# MTCO2 cover

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from c3s_eqc_automatic_quality_control import download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
start = "2007-07"
stop = "2024-12"

## Define request

In [ ]:
collection_id = "satellite-carbon-dioxide"
request = {
    "processing_level": ["level_3"],
    "variable": "co2",
    "sensor_and_algorithm": ["iasi_merged_obs4mips"],
    "version": ["10_1"],
}
requests = download.update_request_date(request, start, stop, stringify_dates=True)

## Define functions to cache

In [ ]:
def compute_coverage(da, dim="time"):
    return (da > 0).sum(dim) / da.sizes[dim]


def compute_mtco2_coverage(ds):
    da = ds["mtco2_nobs"].groupby("time.month").map(compute_coverage)
    da.attrs = {"long_name": "MTCO$_2$ Data Coverage"}
    return da.to_dataset(name="mtco2_coverage")

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    chunks={"year": 1, "month": 1},
    transform_func=compute_mtco2_coverage,
    transform_chunks=False,
)

## Quick and dirty plot

In [ ]:
facet = plot.projected_map(
    ds["mtco2_coverage"],
    col="month",
    col_wrap=3,
    cmap="Greens",
    cbar_kwargs={"orientation": "horizontal"},
    levels=np.linspace(0, 1.01, 11),
    vmax=1.0,
)
facet.cbar.ax.xaxis.set_major_formatter(lambda x, pos: f"{x:.1f}")